In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.autograd as autograd

In [2]:
class LSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.hidden2out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.hidden = self.initHidden()
    
    def forward(self, sequence):
        x = sequence.view(len(sequence), self.batch_size , -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2out(lstm_out[-1])
        output = self.softmax(y)
        return output

    def initHidden(self):
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_size)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_size)))

In [3]:
learning_rate = 0.001
criterion = nn.NLLLoss()

def train(y_train, x_train, rnn):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    output = rnn(x_train)

    loss = criterion(output, y_train)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        gd = p.grad.data
        gd[gd != gd] = 0
        p.data.add_(-learning_rate, gd)

    return output, loss.item()